We want to first explore the function make_a_obs_profile_y in poc.py, it has fbeam intake, which is the interpolated beam profile. Thus very likely, it will convolve the beam with soem profile. 

In [ ]:
def make_a_obs_profile_y(thta_arc,M,z,theta_0,fbeam,ybeam):
    rho = np.zeros(len(thta_arc))
    pth = np.zeros(len(thta_arc))
    for ii in range(len(thta_arc)):
        temp = project_prof_beam_y(thta_arc[ii],M,z,theta_0,fbeam,ybeam)
        rho[ii] = temp[0]
        pth[ii] = temp[1]
    return rho,pth

Known from Stefania code: 
- *thta-arc -- very likely the degree range in arcmin
- M -- M_200
- z -- ofc
- ***theta_0 -- tricky one, we know it takes 5 parameters, very unsure
- fbeam -- interpolated beam in multipole expansioin, function looks like

In [ ]:
def fBeamF_150(x):
    return np.interp(x,ell,beam_150_ell) 

-  *ybeam -- likely the FWH of gaussian beam, but which one?
-  rho -- is the gas density parameter of the GNFW profile.
-  pth -- the pressure profile

# now let's go explore the function project_prof_beam_y

In [ ]:
def project_prof_beam_y(tht,M,z,theta,f_beam,y_beam):
    disc_fac = np.sqrt(2)
    l0 = 30000.
    NNR = 100
    resolution_factor = 2.0
    NNR2 = resolution_factor*NNR

    fwhm = y_beam
    fwhm *= np.pi / (180.*60.)
    sigmaBeam = fwhm / np.sqrt(8.*np.log(2.))

    #P0, rho0, x_f = theta2
    #fstar = fstar_func(M)
    theta_r = [theta[0],theta[1],theta[2],theta[3]]
    theta_p = [theta[0],theta[1],theta[2],theta[4]]

: 

Known:
- disc_fac is AP
- *l0 is the size of beam ell? 
- *NNR: number of radial bins, and NNR2 is parametrized with resolution facor

Equations: 
- converting beam from arcminute to radians amd sigma is the equation for standard deviation
$$ \text{FWHM} = y_{\text{beam}} \times \frac{\pi}{180 \times 60}$$

$$\sigma_{\text{Beam}} = \frac{\text{FWHM}}{\sqrt{8 \ln 2}} $$
Questions are: 
- what y_beam fwhm do? the gaussian beam we convolved with?
- $\color{red}{\text{Buggy!}}$ Theta _r which is for density profile and Theta_p (param for pressure) taking different values of theta_0 or theta array

In [ ]:
   AngDis = AngDist(z)

    rvir = r200(M,z)/kpc_cgs/1e3 #in MPC
    c = con(M,z)

    r_ext = AngDis*np.arctan(np.radians(tht/60.))
    r_ext2 = AngDis*np.arctan(np.radians(tht*disc_fac/60.))

    rad = np.logspace(-3, 1, 200) #in MPC
    rad2 = np.logspace(-3, 1, 200) #in MPC

    radlim = r_ext
    radlim2 = r_ext2

    dtht = np.arctan(radlim/AngDis)/NNR # rads
    dtht2 = np.arctan(radlim2/AngDis)/NNR # rads

    thta = (np.arange(NNR) + 1.)*dtht
    thta2 = (np.arange(NNR) + 1.)*dtht2

    thta_smooth = (np.arange(NNR2) + 1.)*dtht/resolution_factor
    thta2_smooth = (np.arange(NNR2) + 1.)*dtht2/resolution_factor

: 

- Calculating the viral radius and kpc_cgs is kpc in cm somehow 
$$ r_{\text{vir}} = \frac{r_{200}(M, z)}{1\, \text{kpc in cm}} \times 10^{-3} \, \text{Mpc}
$$
con(M,z) is the function from obb.py returns concentration parameter from Duffy et al. (2008)\\
r_ext and r_ext2 are the inner and outter AP physical size of the object. 
$$
\theta radian = \text{tht} \times \frac{\pi}{180 \times 60} \\

r_{\text{ext}} = D_A(z) \arctan(\theta)    ~~~in Mpc
$$
$\color{red}{\text{Why arctan not tan? Using arctan means it converts from physical size of the object to the distance}}$

- rad and rad2 are same size log space from e-3 to e1 also in physical distance Mpc
- rad log space is bounded by r_ext and r_ext2 which is the AP ring of the galaxy
- dtht and dtht2 are numerical steps for inner and outter bound in radians, smoothness defined by NNR
$$ d\theta = \frac{\arctan\left( \frac{r_{\text{lim}}}{D_A(z)} \right)}{NNR}
$$
- thta and thta2 are step x coordinates or index? start with 1
- thta_smooth achieve the same nax and min of dtha list but the increment is proportional to 1/resolution factor.


In [9]:
import numpy as np 
bba = (np.arange(200) + 1.)*0.2/2
bba

array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,
        5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,
        6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,  7.7,
        7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,  8.8,
        8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,  9.9,
       10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11. ,
       11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. , 12.1,
       12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1, 13.2,
       13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2, 14.3,
       14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15. , 15

In [10]:
abb = bba[:,None]

In [16]:
abb[1]

IndexError: index 1 is out of bounds for axis 0 with size 1